#Titanic: Machine Learning from Disaster
Olivier RISSER-MAROIX (VieVie31)

In [1]:
import graphlab as gl

A newer version of GraphLab Create (v1.8.5) is available! Your current version is v1.8.4.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


In [2]:
data_train = gl.load_sframe("train.csv")
data_test = gl.load_sframe("test.csv")

2016-03-19 09:26:18,885 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.4 started. Logging: /tmp/graphlab_server_1458375975.log


Finished parsing file /Users/mac/Documents/Programmation/Kaggle/titanic/train.csv

Parsing completed. Parsed 100 lines in 0.029445 secs.

This non-commercial license of GraphLab Create is assigned to orissermaroix@gmail.com and will expire on March 12, 2017. For commercial licensing options, visit https://dato.com/buy/.
------------------------------------------------------

Finished parsing file /Users/mac/Documents/Programmation/Kaggle/titanic/train.csv

...

Parsing completed. Parsed 100 lines in 0.009108 secs.


Inferred types from first line of file as 
column_type_hints=[int,int,int,str,str,float,int,int,str,float,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
------------------------------------------------------

Finished parsing file /Users/mac/Documents/Programmation/Kaggle/titanic/test.csv

Parsing completed. Parsed 418 lines in 0.011496 secs.


Inferred types from first line of file as 
column_type_hints=[int,int,str,str,float,int,int,str,float,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


In [3]:
data_train.head(3)

Columns:
	PassengerId	int
	Survived	int
	Pclass	int
	Name	str
	Sex	str
	Age	float
	SibSp	int
	Parch	int
	Ticket	str
	Fare	float
	Cabin	str
	Embarked	str

Rows: 3

Data:
+-------------+----------+--------+-------------------------------+--------+
| PassengerId | Survived | Pclass |              Name             |  Sex   |
+-------------+----------+--------+-------------------------------+--------+
|      1      |    0     |   3    |    Braund, Mr. Owen Harris    |  male  |
|      2      |    1     |   1    | Cumings, Mrs. John Bradley... | female |
|      3      |    1     |   3    |     Heikkinen, Miss. Laina    | female |
+-------------+----------+--------+-------------------------------+--------+
+------+-------+-------+------------------+---------+-------+----------+
| Age  | SibSp | Parch |      Ticket      |   Fare  | Cabin | Embarked |
+------+-------+-------+------------------+---------+-------+----------+
| 22.0 |   1   |   0   |    A/5 21171     |   7.25  |       |    S     |
| 38.0 |   1   |   0   |     PC 17599     | 71.2833 |  C85  |    C     |
| 26.0 |   0   |   0   | STON/O2. 3101282 |  7.925  |       |    S     |
+------+-------+-------+------------------+---------+-------+----------+
[3 rows x 12 columns]

##Cleanning trainning data

In [4]:
data_train["male"] = data_train["Sex"] == "male"
data_train["female"] = data_train["Sex"] == "female"
data_train = data_train.remove_column("Sex")

In [5]:
data_train["no_age"] = data_train["Age"] == None
data_train["Age"] = gl.SArray([0 if v == None else v for v in data_train["Age"]])

In [6]:
data_train["embarked_s"] = data_train["Embarked"] == "S"
data_train["embarked_c"] = data_train["Embarked"] == "C"
data_train["embarked_q"] = data_train["Embarked"] == "Q"
data_train["embarked_none"] = data_train["Embarked"] == None
data_train = data_train.remove_column("Embarked")

In [7]:
data_train["1_class"] = data_train["Pclass"] == 1
data_train["2_class"] = data_train["Pclass"] == 2
data_train["3_class"] = data_train["Pclass"] == 3
data_train = data_train.remove_column("Pclass")

In [8]:
gl.canvas.set_target("ipynb")
print data_train.head(3)
print data_train["Ticket"]
for v in data_train["Ticket"]:
    print v, " ",

+-------------+----------+-------------------------------+------+-------+-------+
| PassengerId | Survived |              Name             | Age  | SibSp | Parch |
+-------------+----------+-------------------------------+------+-------+-------+
|      1      |    0     |    Braund, Mr. Owen Harris    | 22.0 |   1   |   0   |
|      2      |    1     | Cumings, Mrs. John Bradley... | 38.0 |   1   |   0   |
|      3      |    1     |     Heikkinen, Miss. Laina    | 26.0 |   0   |   0   |
+-------------+----------+-------------------------------+------+-------+-------+
+------------------+---------+-------+------+--------+--------+------------+------------+
|      Ticket      |   Fare  | Cabin | male | female | no_age | embarked_s | embarked_c |
+------------------+---------+-------+------+--------+--------+------------+------------+
|    A/5 21171     |   7.25  |       |  1   |   0    |   0    |     1      |     0      |
|     PC 17599     | 71.2833 |  C85  |  0   |   1    |   0    |   

In [9]:
#processing the tickets numbers
#for try remove the non alpha numerics numbers
def toNumber(string):
    s = "0"
    for v in string:
        if v in ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]:
            s += v
    return int(s)

data_train["PC"] = gl.SArray(["PC" in v for v in data_train["Ticket"]])
data_train["CA"] = gl.SArray(["CA" in v for v in data_train["Ticket"]])
data_train["C.A."] = gl.SArray(["C.A." in v for v in data_train["Ticket"]])
data_train["W./C."] = gl.SArray(["W./C." in v for v in data_train["Ticket"]])
data_train["SOTON"] = gl.SArray(["SOTON" in v for v in data_train["Ticket"]])
data_train["number"] = gl.SArray([toNumber(v) for v in data_train["Ticket"]])
data_train.head(3)

Columns:
	PassengerId	int
	Survived	int
	Name	str
	Age	float
	SibSp	int
	Parch	int
	Ticket	str
	Fare	float
	Cabin	str
	male	int
	female	int
	no_age	int
	embarked_s	int
	embarked_c	int
	embarked_q	int
	embarked_none	int
	1_class	int
	2_class	int
	3_class	int
	PC	int
	CA	int
	C.A.	int
	W./C.	int
	SOTON	int
	number	int

Rows: 3

Data:
+-------------+----------+-------------------------------+------+-------+-------+
| PassengerId | Survived |              Name             | Age  | SibSp | Parch |
+-------------+----------+-------------------------------+------+-------+-------+
|      1      |    0     |    Braund, Mr. Owen Harris    | 22.0 |   1   |   0   |
|      2      |    1     | Cumings, Mrs. John Bradley... | 38.0 |   1   |   0   |
|      3      |    1     |     Heikkinen, Miss. Laina    | 26.0 |   0   |   0   |
+-------------+----------+-------------------------------+------+-------+-------+
+------------------+---------+-------+------+--------+--------+------------+------------+
|      Ticket      |   Fare  | Cabin | male | female | no_age | embarked_s | embarked_c |
+------------------+---------+-------+------+--------+--------+------------+------------+
|    A/5 21171     |   7.25  |       |  1   |   0    |   0    |     1      |     0      |
|     PC 17599     | 71.2833 |  C85  |  0   |   1    |   0    |     0      |     1      |
| STON/O2. 3101282 |  7.925  |       |  0   |   1    |   0    |     1      |     0      |
+------------------+---------+-------+------+--------+--------+------------+------------+
+------------+---------------+---------+---------+---------+----+-----+
| embarked_q | embarked_none | 1_class | 2_class | 3_class | PC | ... |
+------------+---------------+---------+---------+---------+----+-----+
|     0      |       0       |    0    |    0    |    1    | 0  | ... |
|     0      |       0       |    1    |    0    |    0    | 1  | ... |
|     0      |       0       |    0    |    0    |    1    | 0  | ... |
+------------+---------------+---------+---------+---------+----+-----+
[3 rows x 25 columns]

In [10]:
import re

civilite_pattern = re.compile(r" ([A-Za-z])+.")

def get_civilite(name):
    try:
        return civilite_pattern.search(name).group(0)
    except: 
        return ""

civilites_lst = set([get_civilite(v) for v in data_train["Name"]])
print civilites_lst

for c in civilites_lst:
    data_train[c] = gl.SArray([get_civilite(v) == c for v in data_train["Name"]])

data_train.head(1)

set([' Messemaeker,', ' Capt.', ' Col.', ' Pelsmaeker,', ' Melkebeke,', ' Steen,', ' Miss.', ' Master.', ' Rev.', ' y ', ' Mulder,', ' the ', ' Mrs.', ' Velde,', ' Don.', ' Ms.', ' Billiard,', ' Mme.', ' Major.', ' Cruyssen,', ' Mlle.', ' Jonkheer.', ' Carlo,', ' Planke,', ' Walle,', ' Shawah,', ' Dr.', ' Impe,', ' Mr.', ' Gordon,', ' der '])


Columns:
	PassengerId	int
	Survived	int
	Name	str
	Age	float
	SibSp	int
	Parch	int
	Ticket	str
	Fare	float
	Cabin	str
	male	int
	female	int
	no_age	int
	embarked_s	int
	embarked_c	int
	embarked_q	int
	embarked_none	int
	1_class	int
	2_class	int
	3_class	int
	PC	int
	CA	int
	C.A.	int
	W./C.	int
	SOTON	int
	number	int
	 Messemaeker,	int
	 Capt.	int
	 Col.	int
	 Pelsmaeker,	int
	 Melkebeke,	int
	 Steen,	int
	 Miss.	int
	 Master.	int
	 Rev.	int
	 y 	int
	 Mulder,	int
	 the 	int
	 Mrs.	int
	 Velde,	int
	 Don.	int
	 Ms.	int
	 Billiard,	int
	 Mme.	int
	 Major.	int
	 Cruyssen,	int
	 Mlle.	int
	 Jonkheer.	int
	 Carlo,	int
	 Planke,	int
	 Walle,	int
	 Shawah,	int
	 Dr.	int
	 Impe,	int
	 Mr.	int
	 Gordon,	int
	 der 	int

Rows: 1

Data:
+-------------+----------+-------------------------+------+-------+-------+
| PassengerId | Survived |           Name          | Age  | SibSp | Parch |
+-------------+----------+-------------------------+------+-------+-------+
|      1      |    0     | Braund, Mr. Owen Harris | 22.0 |   1   |   0   |
+-------------+----------+-------------------------+------+-------+-------+
+-----------+------+-------+------+--------+--------+------------+------------+
|   Ticket  | Fare | Cabin | male | female | no_age | embarked_s | embarked_c |
+-----------+------+-------+------+--------+--------+------------+------------+
| A/5 21171 | 7.25 |       |  1   |   0    |   0    |     1      |     0      |
+-----------+------+-------+------+--------+--------+------------+------------+
+------------+---------------+---------+---------+---------+----+-----+
| embarked_q | embarked_none | 1_class | 2_class | 3_class | PC | ... |
+------------+---------------+---------+---------+---------+----+-----+
|     0      |       0       |    0    |    0    |    1    | 0  | ... |
+------------+---------------+---------+---------+---------+----+-----+
[1 rows x 56 columns]

In [11]:
train_set_1, train_set_2 = data_train.random_split(.8)

In [12]:
print train_set_1.head(1)
features = ["Age", "SibSp", "Parch", "Fare", "male", "female", "no_age", 
            "embarked_s", "embarked_c", "embarked_q", "embarked_none",
            "1_class", "2_class", "3_class", "CA", "C.A.", "W./C.", "SOTON"] + list(civilites_lst) #, "number"]

+-------------+----------+-------------------------+------+-------+-------+
| PassengerId | Survived |           Name          | Age  | SibSp | Parch |
+-------------+----------+-------------------------+------+-------+-------+
|      1      |    0     | Braund, Mr. Owen Harris | 22.0 |   1   |   0   |
+-------------+----------+-------------------------+------+-------+-------+
+-----------+------+-------+------+--------+--------+------------+------------+
|   Ticket  | Fare | Cabin | male | female | no_age | embarked_s | embarked_c |
+-----------+------+-------+------+--------+--------+------------+------------+
| A/5 21171 | 7.25 |       |  1   |   0    |   0    |     1      |     0      |
+-----------+------+-------+------+--------+--------+------------+------------+
+------------+---------------+---------+---------+---------+----+-----+
| embarked_q | embarked_none | 1_class | 2_class | 3_class | PC | ... |
+------------+---------------+---------+---------+---------+----+-----+
|   

##Create logistic model

In [13]:
#help(gl.classifier.logistic_classifier.create)

In [14]:
simple_logistic_classifier = gl.classifier.logistic_classifier.create(train_set_1, target="Survived", 
                                                                      features=features, validation_set=train_set_2)

WARNING: Detected extremely low variance for feature(s) 'embarked_none', ' Carlo,', ' Walle,', ' der ' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Logistic regression:

--------------------------------------------------------

Number of examples          : 707

Number of classes           : 2

...

| 4         | 5        | 1.067973     | 0.824611          | 0.875000            |

| 5         | 6        | 1.076869     | 0.824611          | 0.875000            |

| 6         | 7        | 1.084083     | 0.824611          | 0.875000            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

##Create SVM model

In [15]:
simple_svm_classifier = gl.classifier.svm_classifier.create(train_set_1, target="Survived", 
                                                            features=features, validation_set=train_set_2, 
                                                            max_iterations=1000)

WARNING: Detected extremely low variance for feature(s) 'embarked_none', ' Carlo,', ' Walle,', ' der ' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

SVM:

--------------------------------------------------------

Number of examples          : 707

Number of classes           : 2

...

| 11        | 16       | 1.000000  | 0.049450     | 0.821782          | 0.864130            |

| 51        | 76       | 0.250000  | 0.220206     | 0.826025          | 0.875000            |

| 101       | 162      | 1.000000  | 0.436872     | 0.826025          | 0.875000            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

##Create a decision tree model

In [16]:
decision_tree_model = gl.decision_tree_classifier.create(train_set_1, validation_set=train_set_2,
                                                               target="Survived", features=features)

WARNING: Detected extremely low variance for feature(s) 'embarked_none', ' Carlo,', ' Walle,', ' der ' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Decision tree classifier:

--------------------------------------------------------

Number of examples          : 707

Number of classes           : 2

...

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

| Iteration | Elapsed Time | Training-accuracy | Training-log_loss | Validation-accuracy | Validation-log_loss |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

| 1         | 0.031944     | 0.864215          | 0.553429          | 0.858696            | 0.555982            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

##Boosted Tree model

In [17]:
boosted_tree_model = gl.classifier.boosted_trees_classifier.create(train_set_1, validation_set=train_set_2,
                                                                   target="Survived", features=features)

WARNING: Detected extremely low variance for feature(s) 'embarked_none', ' Carlo,', ' Walle,', ' der ' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 707

Number of classes           : 2

...

| 3         | 0.020989     | 0.879774          | 0.417142          | 0.858696            | 0.429650            |

| 4         | 0.026557     | 0.899576          | 0.376210          | 0.836957            | 0.405464            |

| 5         | 0.032727     | 0.910891          | 0.340576          | 0.836957            | 0.387609            |

| 6         | 0.038748     | 0.910891          | 0.318386          | 0.831522            | 0.372499            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

##Random Forest model

In [18]:
random_forest_model = gl.classifier.random_forest_classifier.create(train_set_1, validation_set=train_set_2,
                                                                    target="Survived", features=features)

WARNING: Detected extremely low variance for feature(s) 'embarked_none', ' Carlo,', ' Walle,', ' der ' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Random forest classifier:

--------------------------------------------------------

Number of examples          : 707

Number of classes           : 2

...

| 3         | 0.019803     | 0.881188          | 0.553966          | 0.836957            | 0.562002            |

| 4         | 0.024049     | 0.884017          | 0.519364          | 0.842391            | 0.525393            |

| 5         | 0.028578     | 0.885431          | 0.488104          | 0.836957            | 0.494696            |

| 6         | 0.033716     | 0.886846          | 0.458547          | 0.842391            | 0.467322            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

##Cleanning testing data
DO NOT FORGOT ANY CLEANING OPERATION MADE IN THE TRAINING DATA AND USED BY CLASSIFIER !!!

In [19]:
data_test["male"] = data_test["Sex"] == "male"
data_test["female"] = data_test["Sex"] == "female"
data_test = data_test.remove_column("Sex")

In [20]:
data_test["no_age"] = data_test["Age"] == None
data_test["Age"] = gl.SArray([0 if v == None else v for v in data_test["Age"]])

In [21]:
data_test["embarked_s"] = data_test["Embarked"] == "S"
data_test["embarked_c"] = data_test["Embarked"] == "C"
data_test["embarked_q"] = data_test["Embarked"] == "Q"
data_test["embarked_none"] = data_test["Embarked"] == None
data_test = data_test.remove_column("Embarked")

In [22]:
data_test["1_class"] = data_test["Pclass"] == 1
data_test["2_class"] = data_test["Pclass"] == 2
data_test["3_class"] = data_test["Pclass"] == 3
data_test = data_test.remove_column("Pclass")

In [23]:
data_test["number"] = gl.SArray([toNumber(v) for v in data_test["Ticket"]])
data_test["PC"] = gl.SArray(["PC" in v for v in data_test["Ticket"]])
data_test["CA"] = gl.SArray(["CA" in v for v in data_test["Ticket"]])
data_test["C.A."] = gl.SArray(["C.A." in v for v in data_test["Ticket"]])
data_test["W./C."] = gl.SArray(["W./C." in v for v in data_test["Ticket"]])
data_test["SOTON"] = gl.SArray(["SOTON" in v for v in data_test["Ticket"]])
data_test["number"] = gl.SArray([toNumber(v) for v in data_test["Ticket"]])

In [24]:
for c in civilites_lst:
    data_test[c] = gl.SArray([get_civilite(v) == c for v in data_test["Name"]])

##Making Predictions

In [25]:
data_test["Survived"] = random_forest_model.predict(data_test)
#random_forest_model.predict(data_test)
#boosted_tree_model.predict(data_test)
#decision_tree_model.predict(data_test)
#simple_svm_classifier.predict(data_test) 
#simple_logistic_classifier.predict(data_test)

In [26]:
submission = gl.SFrame()

In [27]:
submission["PassengerId"] = data_test["PassengerId"]
submission["Survived"] = data_test["Survived"]

In [28]:
submission.save("kaggle.csv", format="csv")